In [1]:
from deep_mri.dataset import dataset_factory, CLASS_NAMES
from deep_mri.model_zoo import model_factory
import pandas as pd

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
model = model_factory('3d_martin', **{"input_shape" : (97, 115, 97, 1),
                                      "conv_layers": 2,
                                      "conv_kernel" : 3,
                                      "init_filters" : 128})
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 49, 58, 49, 128)   3584      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 25, 29, 25, 256)   884992    
_________________________________________________________________
flatten (Flatten)            (None, 4640000)           0         
_________________________________________________________________
dense (Dense)                (None, 400)               1856000400
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1203      
Total params: 1,856,890,179
Trainable params: 1,856,890,179
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_ds, valid_ds = dataset_factory("3d", True, True, '/home/karelto1/MRI/minc_beast/*/*/*.nii', **{
    "normalize" : True,
    "downscale_ratio" : 2
  })


In [ ]:
batch_size=1
epochs=100
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['AUC', 'accuracy'])

model.fit(train_ds.batch(batch_size),
          epochs=epochs,
          validation_data=valid_ds.batch(batch_size),
          verbose=1,
          workers=40,
          use_multiprocessing=True
          )
